In [1]:
using Pkg
Pkg.activate(@__DIR__);
Pkg.instantiate()

using StatsPlots
using CSV, DataFrames, DataFramesMeta

using Dates
import Base: tryparse

  Activating project at `c:\Users\ayats\Documents\ici-parsing`


In [2]:
@enum VarKind begin
    Hum
    Pres
    Temp
end


function tryparse(::Type{VarKind}, s::String)
    if s == "6"
        Hum::VarKind
    elseif s == "7"
        Pres::VarKind
    elseif s == "11"
        Temp::VarKind
    else
        nothing
    end
end


data = CSV.File(
    open("data/Montecoche_Marzo2023.csv");
    normalizenames=true,
    select=[:ts, :value, :var_id],
    types=Dict(:ts => Dates.DateTime, :var_id => VarKind),
    dateformat="yyyy-mm-dd HH:MM:SS.s"
) |> DataFrame

@transform!(data, @byrow :ts_unix = datetime2unix(:ts))

Row,var_id,value,ts,ts_unix
,VarKind,Float64,DateTime,Float64
1,Hum,76.6,2023-03-01T00:05:01,1.67763e9
2,Hum,84.3,2023-03-01T00:24:56,1.67763e9
3,Hum,87.3,2023-03-01T00:35:20,1.67763e9
4,Hum,100.0,2023-03-01T01:24:33,1.67763e9
5,Hum,100.0,2023-03-01T02:55:42,1.67764e9
6,Hum,100.0,2023-03-01T03:15:01,1.67764e9
7,Hum,100.0,2023-03-01T03:24:33,1.67764e9
8,Hum,100.0,2023-03-01T03:34:51,1.67764e9
9,Hum,95.7,2023-03-01T03:46:02,1.67764e9


In [4]:
using Query

data_hum = data |> 
    @filter(_.var_id == Hum::VarKind) |>
    DataFrame

data_hum |>
    @df plot(:ts, :value)

In [10]:
data_hum = data_hum |> 
    @filter(_.value <= 100.0) |>
    @filter(_.value >= 0.0) |>
    DataFrame

@df data_hum plot(:ts, :value, marker=:circle, markersize=2)

ylims!((-10, 110))

In [12]:
data_hum_resampled = DataFrame(ts=collect(range(
    data_hum[:, :ts] |> minimum,
    data_hum[:, :ts] |> maximum,
    step=Dates.Minute(10)
)))

@transform!(data_hum_resampled, @byrow :ts_unix = datetime2unix(:ts))

Row,ts,ts_unix
,DateTime,Float64
1,2023-03-01T00:05:01,1.67763e9
2,2023-03-01T00:15:01,1.67763e9
3,2023-03-01T00:25:01,1.67763e9
4,2023-03-01T00:35:01,1.67763e9
5,2023-03-01T00:45:01,1.67763e9
6,2023-03-01T00:55:01,1.67763e9
7,2023-03-01T01:05:01,1.67763e9
8,2023-03-01T01:15:01,1.67763e9
9,2023-03-01T01:25:01,1.67763e9


In [22]:
import BasicInterpolators as BI

interpolator = BI.LinearInterpolator(data_hum[:, :ts_unix], data_hum[:, :value])

@transform!(data_hum_resampled, @byrow :value = interpolator(:ts_unix))

display(data_hum_resampled)
@df data_hum_resampled plot(:ts, :value, marker=:circle, markersize=1)
plot!(size=(1200,400))

Row,ts,ts_unix,value
,DateTime,Float64,Float64
1,2023-03-01T00:05:01,1.67763e9,76.6
2,2023-03-01T00:15:01,1.67763e9,80.4661
3,2023-03-01T00:25:01,1.67763e9,84.324
4,2023-03-01T00:35:01,1.67763e9,87.2087
5,2023-03-01T00:45:01,1.67763e9,89.7987
6,2023-03-01T00:55:01,1.67763e9,92.3791
7,2023-03-01T01:05:01,1.67763e9,94.9596
8,2023-03-01T01:15:01,1.67763e9,97.54
9,2023-03-01T01:25:01,1.67763e9,100.0
